In [12]:
# Lire le fichier contenant les paramètres
df = spark.read.option("header", "true").csv("Files/tmp/params_sources")
sources_arg = df.collect()[0]["sources"]
sources_to_process = sources_arg.split(",")

print(f"📥 Sources à traiter : {sources_to_process}")

# 👉 PAS de return ici
result = "OK"


StatementMeta(, 222cf206-6871-4ef3-9a9d-dd15395ebde3, 21, Finished, Available, Finished)

AnalysisException: [PATH_NOT_FOUND] Path does not exist: abfss://f41eb3f0-6e0e-4c57-bd9a-3abe27071266@onelake.dfs.fabric.microsoft.com/fcadd4eb-b2af-422b-b234-f9838415580c/Files/tmp/params_sources.

In [5]:
import yaml
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from notebookutils import mssparkutils
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, LongType, ShortType, TimestampType
from datetime import datetime
import uuid

StatementMeta(, 222cf206-6871-4ef3-9a9d-dd15395ebde3, 7, Finished, Available, Finished)

In [6]:
yaml_file_paths = [
    "abfss://test@onelake.dfs.fabric.microsoft.com/lakehouse_bronze.Lakehouse/Files/employee/employee.yaml",
    "abfss://test@onelake.dfs.fabric.microsoft.com/lakehouse_bronze.Lakehouse/Files/education_level/education_level.yaml",
    "abfss://test@onelake.dfs.fabric.microsoft.com/lakehouse_bronze.Lakehouse/Files/performance_rating/performance_rating.yaml",
    "abfss://test@onelake.dfs.fabric.microsoft.com/lakehouse_bronze.Lakehouse/Files/rating_level/rating_level.yaml",
    "abfss://test@onelake.dfs.fabric.microsoft.com/lakehouse_bronze.Lakehouse/Files/satisfied_level/satisfied_level.yaml"
]

configs = []

for yaml_file_path in yaml_file_paths:
    yaml_content = "\n".join([row["value"] for row in spark.read.format("text").load(yaml_file_path).collect()])
    config = yaml.safe_load(yaml_content)
    configs.append(config)

final_config = {"tables": []}
for config in configs:
    final_config["tables"].extend(config["tables"])

StatementMeta(, 222cf206-6871-4ef3-9a9d-dd15395ebde3, 8, Finished, Available, Finished)

In [7]:
%run "notebook_function"

StatementMeta(, 222cf206-6871-4ef3-9a9d-dd15395ebde3, 16, Finished, Available, Finished)

In [8]:


file_format = "csv"
lakehouse = "lakehouse_bronze"
notebook = "notebook_load_data"
load_data(final_config,lakehouse,file_format,notebook, sources_to_process=sources_to_process)

StatementMeta(, 222cf206-6871-4ef3-9a9d-dd15395ebde3, 17, Finished, Available, Finished)


🚀 Traitement du dossier source : Files/employee/inbound/ pour la table employee
📂 Lecture du fichier : Employee.csv
✅ 1470 lignes chargées depuis Employee.csv
💾 Données valides écrites dans lakehouse_bronze.employee
✅ Log ajouté avec succès pour abfss://test@onelake.dfs.fabric.microsoft.com/lakehouse_bronze.Lakehouse/Files/employee/inbound//Employee.csv
📝 Log flow ajouté pour Employee.csv - Flow ID : 5931309818
 Log ajouté avec succès pour employee
📌 Log job ajouté pour employee - Statut : SUCCESS

🚀 Traitement du dossier source : Files/education_level/inbound/ pour la table education_level
📂 Lecture du fichier : EducationLevel.csv
✅ 5 lignes chargées depuis EducationLevel.csv
💾 Données valides écrites dans lakehouse_bronze.education_level
✅ Log ajouté avec succès pour abfss://test@onelake.dfs.fabric.microsoft.com/lakehouse_bronze.Lakehouse/Files/education_level/inbound//EducationLevel.csv
📝 Log flow ajouté pour EducationLevel.csv - Flow ID : 9767744090
 Log ajouté avec succès pour ed

In [17]:
notebook_name = "notebook_load_data"
mode = "SCD1"

insert_bronze_to_silver(final_config, mode, notebook_name)


StatementMeta(, 7c90efe7-3500-404c-bdc9-8f6625feb140, 26, Finished, Available, Finished)


🔄 Transfert de lakehouse_bronze.employee → lakehouse_silver.employee
🔄 Colonnes transformées : ['employee_id', 'first_name', 'last_name', 'gender', 'age', 'business_travel', 'department', 'distance_from_home', 'state', 'ethnicity', 'education', 'education_field', 'job_role', 'marital_status', 'salary', 'stock_option_level', 'over_time', 'hire_date', 'attrition', 'years_at_company', 'years_in_most_recent_role', 'years_since_last_promotion', 'years_with_curr_manager']
✅ Mode SCD1 : Écrasement des anciennes données dans lakehouse_silver.employee
 Log ajouté avec succès pour employee
📊 Log ajouté pour lakehouse_silver.employee

🔄 Transfert de lakehouse_bronze.education_level → lakehouse_silver.education_level
🔄 Colonnes transformées : ['education_level_id', 'education_level']
✅ Mode SCD1 : Écrasement des anciennes données dans lakehouse_silver.education_level
 Log ajouté avec succès pour education_level
📊 Log ajouté pour lakehouse_silver.education_level

🔄 Transfert de lakehouse_bronze.pe

In [6]:
insert_silver_to_gold()


StatementMeta(, df81e3e1-a929-43e5-b993-e03009281a5e, 8, Finished, Available, Finished)

 Données insérées dans `gold_employee`.
 Données insérées dans `gold_performance_rating`.
